In [4]:
#import vectorbtpro as vbt
from vectorbtpro import *
import os
import numpy as np
import talib
from numba import njit


In [ ]:
pip list

# CONFIGURAÇÃO


In [6]:
vbt.settings.execution.engine = 'serial'

'''
# Configurações válidas de caching e chunking
vbt.settings.caching.update({
    'disable': False,
    'use_cached_accessors': True
})

vbt.settings.execution.update({
    'cache_chunks': True,
    'chunk_cache_dir': "cache_chunks",
    'release_chunk_cache': True
})
'''
vbt.settings.set_theme("dark")

# CARREGAR DADOS


In [ ]:
# Carregando os dados de preços usando SerialEngine
DB_PATH = "odysseus/forex_market.duckdb"
symbols = ['EURUSD', 'GBPUSD']
tables = vbt.DuckDBData.list_tables(connection=DB_PATH)
print("Ativos disponíveis:")
print(tables)
data = vbt.DuckDBData.from_duckdb(symbols, start="2013-01-01", connection=DB_PATH)
print(data.close)
print(data.stats())


In [ ]:
%pwd

# RESAMPLE


In [ ]:
time_frames = [1, 4, 24]
freqs = [f'{tf}h' for tf in time_frames]
h1_data = data.resample("1h")
h1_close = h1_data.close
h4_data = data.resample("4h")
h4_close = h4_data.close
h24_data = data.resample("24h")
h24_close = h24_data.close

print(h1_data.stats())
h24_data.data['EURUSD'].vbt.ohlcv.plot().show()
h24_close.vbt.plot().show()

In [ ]:
simbolo = symbols[0]
print(symbols)
h24_data.data[simbolo].vbt.ohlcv.plot().show()

# Indicador de tendência

In [11]:
high = h24_data.high.ffill().bfill()
low = h24_data.low.ffill().bfill()
close = h24_close.ffill().bfill()

In [12]:
def get_basic_bands(med_price, atr, multiplier):
    matr = multiplier * atr
    upper = med_price + matr
    lower = med_price - matr
    return upper, lower

In [13]:
@njit
def get_final_bands_nb(close, upper, lower):
    trend = np.full(close.shape, np.nan)
    dir_ = np.full(close.shape, 1)
    long = np.full(close.shape, np.nan)
    short = np.full(close.shape, np.nan)

    for i in range(1, close.shape[0]):
        if close[i] > upper[i - 1]:
            dir_[i] = 1
        elif close[i] < lower[i - 1]:
            dir_[i] = -1
        else:
            dir_[i] = dir_[i - 1]
            if dir_[i] > 0 and lower[i] < lower[i - 1]:
                lower[i] = lower[i - 1]
            if dir_[i] < 0 and upper[i] > upper[i - 1]:
                upper[i] = upper[i - 1]

        if dir_[i] > 0:
            trend[i] = long[i] = lower[i]
        else:
            trend[i] = short[i] = upper[i]
            
    return trend, dir_, long, short

In [14]:
def faster_supertrend_talib(high, low, close, period=7, multiplier=3):
    avg_price = talib.MEDPRICE(high, low)
    atr = talib.ATR(high, low, close, period)
    upper, lower = get_basic_bands(avg_price, atr, multiplier)
    return get_final_bands_nb(close, upper, lower)

In [ ]:
# Executando o faster_supertrend_talib
trend, dir_, long, short = faster_supertrend_talib(
    high['EURUSD'].values, 
    low['EURUSD'].values, 
    close['EURUSD'].values
)


valid_mask = np.isfinite(trend)
print("Valores válidos do trend:")
print(trend[valid_mask])
print(len(trend[valid_mask]))
print("\nDireção correspondente:")
print(dir_[valid_mask])
print(len(dir_[valid_mask]))
print("\nValores long válidos:")
print(long[np.isfinite(long)])
print(len(long[valid_mask]))
print("\nValores short válidos:")
print(short[np.isfinite(short)])
print(len(short[valid_mask]))

In [16]:
SuperTrend = vbt.IF(
    class_name='SuperTrend',
    short_name='st',
    input_names=['high', 'low', 'close'],
    param_names=['period', 'multiplier'],
    output_names=['supert', 'superd', 'superl', 'supers']
).with_apply_func(
    faster_supertrend_talib, 
    takes_1d=True,
    period=7, 
    multiplier=3
)

In [17]:
class SuperTrend(SuperTrend):
    def plot(self, 
             column=None, 
             close_kwargs=None,
             superl_kwargs=None,
             supers_kwargs=None,
             fig=None, 
             **layout_kwargs):
        close_kwargs = close_kwargs if close_kwargs else {}
        superl_kwargs = superl_kwargs if superl_kwargs else {}
        supers_kwargs = supers_kwargs if supers_kwargs else {}
        
        close = self.select_col_from_obj(self.close, column).rename('Close')
        supers = self.select_col_from_obj(self.supers, column).rename('Short')
        superl = self.select_col_from_obj(self.superl, column).rename('Long')
        
        fig = close.vbt.plot(fig=fig, **close_kwargs, **layout_kwargs)
        supers.vbt.plot(fig=fig, **supers_kwargs)
        superl.vbt.plot(fig=fig, **superl_kwargs)
        
        return fig

## Otimização

In [18]:
periods = np.arange(80, 101)
multipliers = np.arange(50, 71) / 10

In [ ]:
st = SuperTrend.run(
    high, low, close, 
    period=periods, 
    multiplier=multipliers,
    param_product=True,
)
print(st.supert.dropna())

In [ ]:
# Escolha um período e multiplicador específico
periodos = [periods[0], periods[-1]]
multiplicadores = [multipliers[0], multipliers[-1]]

# Criando todas as combinações possíveis
combinacoes = [
    (periodos[0], multiplicadores[0]),  # (4, 2.0)
    (periodos[0], multiplicadores[-1]), # (4, 4.0)
    (periodos[-1], multiplicadores[0]), # (19, 2.0)
    (periodos[-1], multiplicadores[-1]) # (19, 4.0)
]

# Plotando cada combinação separadamente
for periodo, multiplicador in combinacoes:
    st.plot(
        column=(periodo, multiplicador, 'EURUSD'),
        superl_kwargs=dict(trace_kwargs=dict(line_color='limegreen')),
        supers_kwargs=dict(trace_kwargs=dict(line_color='red')),
        title=f'SuperTrend - Período: {periodo}, Multiplicador: {multiplicador}'
    ).show()

# Backtesting

In [21]:
class SuperTrend(SuperTrend):
    def plot(self, 
             column=None, 
             close_kwargs=None,
             superl_kwargs=None,
             supers_kwargs=None,
             plot_signals=True,
             pf=None,
             fig=None, 
             **layout_kwargs):
        # Remove plot_signals dos layout_kwargs para não causar erro
        if 'plot_signals' in layout_kwargs:
            del layout_kwargs['plot_signals']
            
        close_kwargs = close_kwargs if close_kwargs else {}
        superl_kwargs = superl_kwargs if superl_kwargs else {}
        supers_kwargs = supers_kwargs if supers_kwargs else {}
        
        close = self.select_col_from_obj(self.close, column).rename('Close')
        supers = self.select_col_from_obj(self.supers, column).rename('Short')
        superl = self.select_col_from_obj(self.superl, column).rename('Long')
        
        # Cria o gráfico base
        fig = close.vbt.plot(fig=fig, **close_kwargs, **layout_kwargs)
        
        # Adiciona as linhas do SuperTrend
        supers.vbt.plot(fig=fig, **supers_kwargs)
        superl.vbt.plot(fig=fig, **superl_kwargs)
        
        # Adiciona os sinais de entrada/saída se solicitado
        if plot_signals and pf is not None:
            fig = pf.plot_trade_signals(column=column, fig=fig)
        
        return fig

In [22]:
entries = (~st.superl.isnull()).vbt.signals.fshift()
exits = (~st.supers.isnull()).vbt.signals.fshift()

In [23]:
pf = vbt.Portfolio.from_signals(close, entries, exits, init_cash=100, fees=0.001, freq='1D')

In [ ]:
print(symbols)
pf.final_value.vbt.heatmap(
    x_level='st_period', 
    y_level='st_multiplier',
    slider_level='symbol'
).show()

In [ ]:
vbt.Portfolio.from_holding(close, freq='1D').final_value

In [ ]:
st = SuperTrend.run(
    high, low, close, 
    period=91, 
    multiplier=6.1,
    param_product=True,
)
print(st.supert.dropna())

In [ ]:
entries = (~st.superl.isnull()).vbt.signals.fshift()
exits = (~st.supers.isnull()).vbt.signals.fshift()

print(entries)
print(exits)

In [28]:
pf = vbt.Portfolio.from_signals(close, entries, exits, init_cash=100, fees=0.001, freq='1D')

In [ ]:
periodo = 91
multiplicador = 6.1

# Plotando cada simbolo separadamente
for simbolo in symbols:
    st.plot(
        column=(periodo, multiplicador, simbolo),
        superl_kwargs=dict(trace_kwargs=dict(line_color='limegreen')),
        supers_kwargs=dict(trace_kwargs=dict(line_color='red')),
        title=f'SuperTrend - Período: {periodo}, Multiplicador: {multiplicador}'
    ).show()

#Separar a série quando o atr é 1 e quando é -1

In [30]:
alta = (~st.superl.isnull()).vbt.signals.fshift()
baixa = (~st.supers.isnull()).vbt.signals.fshift()

In [ ]:
# Reorganizar os sinais para facilitar o acesso
alta_mask = alta.loc[:, (periodo, multiplicador, slice(None))].droplevel([0,1], axis=1)
baixa_mask = baixa.loc[:, (periodo, multiplicador, slice(None))].droplevel([0,1], axis=1)

for simbolo in symbols:
    # Dados OHLC para tendência de alta
    alta_ohlc = h24_data.data[simbolo].copy()
    alta_ohlc.loc[~alta_mask[simbolo]] = np.nan
    
    # Dados OHLC para tendência de baixa
    baixa_ohlc = h24_data.data[simbolo].copy()
    baixa_ohlc.loc[~baixa_mask[simbolo]] = np.nan

    
    
    # Plotar tendência de alta
    alta_ohlc.vbt.ohlcv.plot(
        title=f'{simbolo} - Períodos de Tendência de Alta'
    ).show()
    
    # Plotar tendência de baixa
    baixa_ohlc.vbt.ohlcv.plot(
        title=f'{simbolo} - Períodos de Tendência de Baixa'
    ).show()

# Resamplear em um timeframe menor, os períodos de alta e de baixa.

In [ ]:
# Criar dicionários para armazenar as séries
alta_1h_dict = {}
baixa_1h_dict = {}
alta_24h_1h_dict = {}
baixa_24h_1h_dict = {}

for simbolo in symbols:
    # Resample do close de 24h para 1h usando realign_closing
    close_24_1h = h24_data.close.vbt.realign_closing("1h")
    
    # Realinhar as máscaras para o mesmo índice do close
    alta_mask_1h = alta_mask[simbolo].vbt.realign(
        close_24_1h[simbolo].index,
        ffill=True
    )
    baixa_mask_1h = baixa_mask[simbolo].vbt.realign(
        close_24_1h[simbolo].index,
        ffill=True
    )
    
    # Plotar comparação dos closes
    fig = h1_close[simbolo].rename("H1").vbt.plot()
    close_24_1h[simbolo].rename("H24_H1").vbt.plot(fig=fig).show()
    
    # Aplicar as máscaras e armazenar nos dicionários
    alta_24h_1h = close_24_1h[simbolo].copy()
    alta_24h_1h[alta_mask_1h.values == False] = np.nan
    alta_24h_1h_dict[simbolo] = alta_24h_1h
    
    baixa_24h_1h = close_24_1h[simbolo].copy()
    baixa_24h_1h[baixa_mask_1h.values == False] = np.nan
    baixa_24h_1h_dict[simbolo] = baixa_24h_1h
    
    # Aplicar máscaras na série H1 e armazenar
    alta_1h = h1_close[simbolo].vbt.realign(close_24_1h[simbolo].index).copy()
    alta_1h[alta_mask_1h.values == False] = np.nan
    alta_1h_dict[simbolo] = alta_1h
    
    baixa_1h = h1_close[simbolo].vbt.realign(close_24_1h[simbolo].index).copy()
    baixa_1h[baixa_mask_1h.values == False] = np.nan
    baixa_1h_dict[simbolo] = baixa_1h
    
    # Plotar comparações
    fig = alta_24h_1h.vbt.plot(title=f'{simbolo} - Alta (Comparação)')
    alta_1h.vbt.plot(fig=fig, trace_kwargs=dict(line_color='gray', opacity=0.7)).show()
    
    fig = baixa_24h_1h.vbt.plot(title=f'{simbolo} - Baixa (Comparação)')
    baixa_1h.vbt.plot(fig=fig, trace_kwargs=dict(line_color='gray', opacity=0.7)).show()

# Agora você pode acessar as séries assim:
print("\nSéries disponíveis para cada símbolo:")
for simbolo in symbols:
    print(f"\n{simbolo}:")
    print(f"Alta 1h: {len(alta_1h_dict[simbolo])} pontos")
    print(f"Baixa 1h: {len(baixa_1h_dict[simbolo])} pontos")

# Encontrar padrões

In [33]:
bullish_patterns = {
    "double_bottom": [5, 1, 3, 1, 5],
    "exp_triangle": [3, 4, 2, 5, 1, 6],
    "asc_triangle": [1, 5, 2, 5, 3, 6],
    "symm_triangle": [1, 6, 2, 5, 3, 6],
    "pennant": [6, 1, 5, 2, 4, 3, 6]
}
bearish_patterns = {
    "head_and_shoulders": [1, 4, 2, 6, 2, 4, 1],
    "double_top": [1, 5, 3, 5, 1],
    "desc_triangle": [6, 2, 5, 2, 4, 1],
    "symm_triangle": [6, 1, 5, 2, 4, 1],
    "pennant": [1, 6, 2, 5, 3, 4, 1]
}


In [ ]:
for bull_pattern in bullish_patterns:
    pd.Series(bullish_patterns[bull_pattern]).vbt.plot(title=f'{bull_pattern} - Padrão de Alta').show()

for bear_pattern in bearish_patterns:
    pd.Series(bearish_patterns[bear_pattern]).vbt.plot(title=f'{bear_pattern} - Padrão de Baixa').show()

In [ ]:
min_window = 8
max_window = 8 * 32

def detect_patterns(patterns, price):
    return vbt.PatternRanges.from_pattern_search(
        price,
        open=h1_data.open,  # OHLC for plotting
        high=h1_data.high,
        low=h1_data.low,
        close=h1_data.close,
        pattern=patterns,
        window=min_window,
        max_window=max_window,
        execute_kwargs=dict(  # multithreading
            engine="threadpool", 
            chunk_len="auto", 
        )
    )

alta_bullish_matches = detect_patterns(vbt.Param(bullish_patterns, name="bullish_pattern"), alta_1h)
alta_bearish_matches = detect_patterns(vbt.Param(bearish_patterns, name="bearish_pattern"), alta_1h)

baixa_bullish_matches = detect_patterns(vbt.Param(bullish_patterns, name="bullish_pattern"), baixa_1h)
baixa_bearish_matches = detect_patterns(vbt.Param(bearish_patterns, name="bearish_pattern"), baixa_1h)

In [ ]:
print(alta_bullish_matches.count())
print(alta_bearish_matches.count())

print(baixa_bullish_matches.count())
print(baixa_bearish_matches.count())

In [ ]:
vbt.settings.plotting.auto_rangebreaks = True  # for stocks

alta_bull_display_column = alta_bullish_matches.count().idxmax()
alta_bear_display_column = alta_bearish_matches.count().idxmax()

baixa_bull_display_column = baixa_bullish_matches.count().idxmax()
baixa_bear_display_column = baixa_bearish_matches.count().idxmax()


alta_bullish_matches.plot(column=alta_bull_display_column, fit_ranges=True).show()
alta_bearish_matches.plot(column=alta_bear_display_column, fit_ranges=True).show()

baixa_bullish_matches.plot(column=baixa_bull_display_column, fit_ranges=True).show()
baixa_bearish_matches.plot(column=baixa_bear_display_column, fit_ranges=True).show()

In [38]:
alta_entries = alta_bullish_matches.last_pd_mask
alta_exits = alta_bearish_matches.last_pd_mask

baixa_entries = baixa_bullish_matches.last_pd_mask
baixa_exits = baixa_bearish_matches.last_pd_mask

In [ ]:
alta_entries, alta_exits = alta_entries.vbt.x(alta_exits)
print(alta_entries.columns)
print(alta_entries)

baixa_entries, baixa_exits = baixa_entries.vbt.x(baixa_exits)
print(baixa_entries.columns)
print(baixa_entries)

In [ ]:
# Criar dicionários para armazenar os resultados
alta_returns = {}
baixa_returns = {}

# Iterar sobre cada combinação de padrões
for bull_pattern in bullish_patterns:
    for bear_pattern in bearish_patterns:
        # Selecionar os sinais para esta combinação específica
        alta_entries_pattern = alta_entries.loc[:, (bull_pattern, bear_pattern)].astype('bool')
        alta_exits_pattern = alta_exits.loc[:, (bull_pattern, bear_pattern)].astype('bool')
        
        baixa_entries_pattern = baixa_entries.loc[:, (bull_pattern, bear_pattern)].astype('bool')
        baixa_exits_pattern = baixa_exits.loc[:, (bull_pattern, bear_pattern)].astype('bool')
        
        # Criar portfolios
        try:
            alta_pf = vbt.Portfolio.from_signals(
                h1_data,
                alta_entries_pattern,
                alta_exits_pattern,
                freq='1h'
            )
            alta_returns[(bull_pattern, bear_pattern)] = alta_pf.total_return  # Removido os parênteses
            
            baixa_pf = vbt.Portfolio.from_signals(
                h1_data,
                baixa_entries_pattern,
                baixa_exits_pattern,
                freq='1h'
            )
            baixa_returns[(bull_pattern, bear_pattern)] = baixa_pf.total_return  # Removido os parênteses
            
        except Exception as e:
            print(f"Erro na combinação {bull_pattern}-{bear_pattern}: {str(e)}")
            continue

# Converter resultados para DataFrame
alta_returns_df = pd.DataFrame.from_dict(alta_returns, orient='index')
baixa_returns_df = pd.DataFrame.from_dict(baixa_returns, orient='index')

print("\nRetornos para padrões de alta:")
print(alta_returns_df)
print("\nRetornos para padrões de baixa:")
print(baixa_returns_df)

In [ ]:
min_window = 8
max_window = 8 * 32

def detect_patterns(patterns, price):
    return vbt.PatternRanges.from_pattern_search(
        price,
        open=data.open,  # OHLC for plotting
        high=data.high,
        low=data.low,
        close=data.close,
        pattern=patterns,
        window=min_window,
        max_window=max_window,
        execute_kwargs=dict(  # multithreading
            engine="threadpool", 
            chunk_len="auto", 
        )
    )

bullish_matches = detect_patterns(vbt.Param(bullish_patterns, name="bullish_pattern"))
bearish_matches = detect_patterns(vbt.Param(bearish_patterns, name="bearish_pattern"))

In [ ]:
# Criar portfolios com broadcasting
alta_pf = vbt.Portfolio.from_signals(
    h1_data,
    entries=alta_entries,
    exits=alta_exits,
    direction='longonly',
    freq='1h',
    broadcast_kwargs=dict(
        columns_from=pd.MultiIndex.from_product([
            bullish_patterns,
            bearish_patterns
        ], names=['bullish_pattern', 'bearish_pattern'])
    )
)

baixa_pf = vbt.Portfolio.from_signals(
    h1_data,
    entries=baixa_entries,
    exits=baixa_exits,
    direction='shortonly',
    freq='1h',
    broadcast_kwargs=dict(
        columns_from=pd.MultiIndex.from_product([
            bullish_patterns,
            bearish_patterns
        ], names=['bullish_pattern', 'bearish_pattern'])
    )
)

# Calcular médias por padrão
alta_mean_return = alta_pf.total_return.groupby(["bullish_pattern", "bearish_pattern"]).mean()
baixa_mean_return = baixa_pf.total_return.groupby(["bullish_pattern", "bearish_pattern"]).mean()

# Plotar heatmaps
alta_mean_return.vbt.heatmap(
    title='Retornos dos Padrões de Alta',
    trace_kwargs=dict(
        xaxis_title='Padrão de Baixa',
        yaxis_title='Padrão de Alta'
    )
).show()

baixa_mean_return.vbt.heatmap(
    title='Retornos dos Padrões de Baixa',
    trace_kwargs=dict(
        xaxis_title='Padrão de Baixa',
        yaxis_title='Padrão de Alta'
    )
).show()

In [ ]:
alta_mean_total_return = alta_pf.total_return.groupby(["bullish_pattern", "bearish_pattern"]).mean()
baixa_mean_total_return = baixa_pf.total_return.groupby(["bullish_pattern", "bearish_pattern"]).mean()

print(alta_mean_total_return)
print(baixa_mean_total_return)

In [ ]:
# Converter os dicionários para DataFrames com índices adequados
alta_returns_df.index = pd.MultiIndex.from_tuples(alta_returns_df.index, names=['bullish_pattern', 'bearish_pattern'])
baixa_returns_df.index = pd.MultiIndex.from_tuples(baixa_returns_df.index, names=['bullish_pattern', 'bearish_pattern'])

# Criar heatmaps
alta_returns_df.unstack().vbt.heatmap(
    title='Retornos dos Padrões de Alta',
    x_label='Padrão de Baixa',
    y_label='Padrão de Alta'
).show()

baixa_returns_df.unstack().vbt.heatmap(
    title='Retornos dos Padrões de Baixa',
    x_label='Padrão de Baixa',
    y_label='Padrão de Alta'
).show()

# Opcional: mostrar também os valores numéricos
print("\nRetornos dos Padrões de Alta:")
print(alta_returns_df.unstack())
print("\nRetornos dos Padrões de Baixa:")
print(baixa_returns_df.unstack())